# 📌 TF-IDF (비슷한 마이룸 추천)

## ✏️ DB 연결

In [10]:
from dotenv import load_dotenv, find_dotenv
import os
import pymysql
import pandas as pd


# 환경변수 불러오기
load_dotenv(find_dotenv())
HOST = os.environ["MYSQL_HOST"] # MySQL host
DB = os.environ["MYSQL_NAME"]   # MySQL name
USER = os.environ["MYSQL_USER"] # MySQL user
PASSWORD = os.environ["MYSQL_PASS"] # MySQL password
PORT = int(os.environ["MYSQL_PORT"]) # MySQL port


# DB 연결
db = pymysql.connect(host=HOST, port=PORT, user=USER, passwd=PASSWORD, db=DB, charset='utf8', autocommit=True, cursorclass=pymysql.cursors.DictCursor)
#  cursor생성
cursor = db.cursor()

### ✏️ 해당 유저의 Tag 조회

In [33]:
tag_sql = "SELECT DISTINCT(`t`.`name`) AS `tags` FROM `tag` AS `t` \
INNER JOIN `arrange` AS `a` ON `t`.`port_seq` = `a`.`port_seq` \
INNER JOIN `room` AS `r` ON `a`.`room_seq` = `r`.`room_seq` \
INNER JOIN `user` AS `u` ON `r`.`user_seq` = `u`.`user_seq` \
WHERE `u`.`user_seq` = 40;"

cursor.execute(tag_sql)
result = cursor.fetchall()

# db.commit()

df_tags = pd.DataFrame(result)
tag_list = list(df_tags["tags"])
tag_list

['node.js', 'express', '아이링크', '백엔드', 'docker', '태그1', '태그2']

### ✏️ 해당 유저의 Portfolio 내용 조회

In [55]:
port_sql = "SELECT DISTINCT(`p`.`summary`) AS `portfolios` FROM `portfolio` AS `p` \
INNER JOIN `arrange` AS `a` ON `p`.`port_seq` = `a`.`port_seq` \
INNER JOIN `room` AS `r` ON `a`.`room_seq` = `r`.`room_seq` \
INNER JOIN `user` AS `u` ON `r`.`user_seq` = `u`.`user_seq` \
WHERE `u`.`user_seq` = 40;"

cursor.execute(port_sql)
df_ports = pd.DataFrame(cursor.fetchall())
port_list = list(df_ports["portfolios"])
port_list

['<h1>아이링크</h1><p>스스로 모든 것을 하고 싶어하는 모든 아이를 위해 우리 서비스는 만들어졌어요.</p>\n<p>아이 혼자서 자신의 하루를 돌아보고 그 날의 기분을 기록해보아요!</p>\n<p>매일 기록한 일상이 선생님과 부모님에게 전달되어 모두의 기억이 될 수 있게 해줄게요.</p>',
 '포트폴리오 내용']

### 포트폴리오 리스트를 텍스트로 변경

In [46]:
port_text = "".join(port_list)
port_text

'<h1>아이링크</h1><p>스스로 모든 것을 하고 싶어하는 모든 아이를 위해 우리 서비스는 만들어졌어요.</p>\n<p>아이 혼자서 자신의 하루를 돌아보고 그 날의 기분을 기록해보아요!</p>\n<p>매일 기록한 일상이 선생님과 부모님에게 전달되어 모두의 기억이 될 수 있게 해줄게요.</p>포트폴리오 내용'

### 형태소 분석

In [47]:
import re

# 한글, 숫자, 영문만 가져옴
def sub_special(s):
    tag_remover = re.compile('<.*?>')
    s = re.sub(tag_remover, '', s)
    return re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣0-9a-zA-Z ]','',s)

In [48]:
port_text = sub_special(port_text)
port_text

'아이링크스스로 모든 것을 하고 싶어하는 모든 아이를 위해 우리 서비스는 만들어졌어요아이 혼자서 자신의 하루를 돌아보고 그 날의 기분을 기록해보아요매일 기록한 일상이 선생님과 부모님에게 전달되어 모두의 기억이 될 수 있게 해줄게요포트폴리오 내용'

In [58]:
from konlpy.tag import Okt
okt = Okt()

file_path = "./stop_words.txt"
with open(file_path, encoding="utf8") as f:
    lines = f.readlines()
stop_words = [line.rstrip("\n") for line in lines]
stop_words

['아',
 '휴',
 '아이구',
 '아이쿠',
 '아이고',
 '어',
 '나',
 '추천',
 '도서',
 '개정',
 '유형',
 '반면',
 '주년',
 '이다',
 '때문',
 '우리가',
 '저희',
 '따라',
 '의해',
 '을',
 '를',
 '에',
 '의',
 '가',
 '으로',
 '로',
 '에게',
 '뿐이다',
 '의거하여',
 '근거하여',
 '입각하여',
 '기준으로',
 '예하면',
 '예를 들면',
 '예를 들자면',
 '저',
 '소인',
 '소생',
 '저희',
 '지말고',
 '하지마',
 '하지마라',
 '다른',
 '물론',
 '또한',
 '그리고',
 '비길수 없다',
 '해서는 안된다',
 '뿐만 아니라',
 '만이 아니다',
 '만은 아니다',
 '막론하고',
 '관계없이',
 '그치지 않다',
 '그러나',
 '그런데',
 '하지만',
 '든간에',
 '논하지 않다',
 '따지지 않다',
 '설사',
 '비록',
 '더라도',
 '아니면',
 '만 못하다',
 '하는 편이 낫다',
 '불문하고',
 '향하여',
 '향해서',
 '향하다',
 '쪽으로',
 '틈타',
 '이용하여',
 '타다',
 '오르다',
 '제외하고',
 '이 외에',
 '이 밖에',
 '하여야',
 '비로소',
 '한다면 몰라도',
 '외에도',
 '이곳',
 '여기',
 '부터',
 '기점으로',
 '따라서',
 '할 생각이다',
 '하려고하다',
 '이리하여',
 '그리하여',
 '그렇게 함으로써',
 '하지만',
 '일때',
 '할때',
 '앞에서',
 '중에서',
 '보는데서',
 '으로써',
 '로써',
 '까지',
 '해야한다',
 '일것이다',
 '반드시',
 '할줄알다',
 '할수있다',
 '할수있어',
 '임에 틀림없다',
 '한다면',
 '등',
 '등등',
 '제',
 '겨우',
 '단지',
 '다만',
 '할뿐',
 '딩동',
 '댕그',
 '대해서',
 '대하여',
 '대하면',
 '훨씬',
 '얼마나',
 '얼마

In [61]:
# 형태소 분석
def morph_and_stopword(data):
    
    text = "".join(list(data["portfolios"]))
    
    # 특수문자 제거
    text = sub_special(text)
    
    #형태소 분석
    words = okt.morphs(text, stem=True)
    
    # 형태소 분석 결과 담을 리스트
    feature = []
    
    #불용어 처리
    for word in words:
        if word not in stop_words and len(word) > 1:
            feature.append(word)

    feat_str = "".join(feature)
    return feature

In [62]:
df_ports.apply(morph_and_stopword, axis=1)

0    [링크, 스스로, 서비스, 만들어지다, 하루, 돌아보다, 기분, 기록, 해보다, 기...
1                                          [포트폴리오, 내용]
dtype: object